### **The Idea**

The idea behind doing freeform analysis is to bring in valuable insights from different angle to the playground. This is an attempt to establish a connection to some excellent EDAs done by the masters here on the Multiple Choice Responses. One good thing about the freeform data, that motivates me to do this analysis, that the responses are open ened. They are 'True Voice' of the respondents. Therefore, they captures several interesting responses that were not considered as an option while designing the survey. Hence, the value.

### **The scope**

The scope, facts and figures of this analysis are restricted to freeform responses only. Since, the data is unstructured, figures may not be exactly accurate or has no significance however trepresents a relative study.

### **The Framework**

The challenge of doing freeform response analysis is that the data is unstructured texts.  First, I started building text clustering model , but dropped the idea in the midway as I realized it will be time consming. Later, I decided to use topic extraction model using Latent Dirichlet Allocation which essentially returned few important topics from all the responses, given a survey question.


> ![](http://i63.tinypic.com/23t3288.jpg)

### **The Analysis**

I have started with the schema. First, filtered out all 'freeform' questions. Then grouped the questions based on the respondents "Asked". For each group, for each question, I feed the responses to the LDA model, which returned me extracted topics from the responses. I carefully observed returned topics, choose relevant keywords to find percentage of responses for them. I have used simple substring search to find pattern and count. Then I proceed to rank order them based on percentage of responses.

### **The Report:** 
[ Reading procedure : Open other EDA's on MCQ's and do a comparative study. You will find many interesting new responses here]

#### **Upcoming tools/technologies** 

Sometimes it is imporant to see what is not there. And Java, C, C++ is not there. They may be the building blocks for the others but they are clearly not the ones people are really looking at.  People expecting more on Neural Network related tools and technologies like **Pytorch, Keras**. Even GPU related **CUDA** by nVIDIA. Or Deep Learning related **Tensorflow, mxnet**. People also focusing more on Big Data related tools like **Scala, H2o**.  **CNTK** may  going to be the game changer for Microsoft. Bayesian's responded with Pymc3.


#### **Upcoming Methodologies** 

**Reinforcement learning** , an area of machine learning inspired by behaviourist psychology**(wiki). General Adversarial Networks(GANS) **are the next big upcoming methods in the AI space.


#### **Upcoming Programming Languages**

Undoubtly **Python** is the winner. Scientific programming languages like **Octave** and scientific computational engine **Wolframe - Alpha** are in the list. You may also like to review newly launched language **Rust**.


#### **Public Data Repositories**

UCI data repository, Package included datasets, Kaggle, books, webscrapping, aws, Numerai, Uniprot, MSCOCO, Twitter, WIkipedia.


#### **Platforms & Resources**

Many interesting platforms came up. **Reddit** seems to be the leader. Seems, **Analytics Vidhya** doing good job in this area. Other, good platforms are **Coursera, Slack, Datacamp, Quora, Udacity, FB, Opendatascience**. Few not so popular platforms like **Datatau** and **Biostars** has also emerged. People are also using **Meetups** as arranged by some of these platforms (like Analytics Vidhya)for learning and collaborative purposes.

**Analytics Vidhya** stands out from the rest as analytics blog. Others are **data science weekly, super data science.**

Kick start with **Courses,Trainings and Bootcamps**. Many are **self taught **used **book, youtube, online courses**.


#### **Focus Areas**

Responses in relation to interpersonal skills - **Domain knowledge, Thinking skills, Communication skills, passion, patience.**
Responses in relation to Tools/Technologies - **Python, excel, scala, SAS, SPSS, Hadoop, Oracle, Linux, Business Tools, Matlab.**
Responses in relation to demonstrated skill sets - **Data analytics project, work industry experience.**
Responses in relation to degree/course - **Phd, Successfully completed course**


#### **Areas of Application**

Classify news/knowledge, image recognition, cancer research, personalization, behaviour prediction, poverty, education, finance, climate change, healthcare and research, robotics, cryptocurrency, video analytics, security, app building

#### **Show case work through**

Personal projects,Thesis, Good publication, certificate, problem solving, kaggle competitons , developed hobby, conference, project papers. project problems, achievements.






I'll continure to write .....

#### **Loading data and required libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation



schema = pd.read_csv("../input/schema.csv")
data_freeform = pd.read_csv("../input/freeformResponses.csv", dtype = object)
data_freeform = data_freeform.dropna(axis = 1, how='all')

#### **Filter out freeform questions**

In [ ]:
schema_freeform = schema[schema['Column'].str.contains("FreeForm")]

#### **Group questions based on 'Asked'**

'Worker' and 'Worker1' is treated as 'Worker' as there is no separate defination found for 'Worker1' in RespondentTypeREADME

In [ ]:
schema_freeform_asked_all = schema_freeform[schema_freeform['Asked'] == 'All']
schema_freeform_asked_codingworker = schema_freeform[schema_freeform['Asked'] == 'CodingWorker']
schema_freeform_asked_codingworkernc = schema_freeform[schema_freeform['Asked'] == 'CodingWorker-NC']
schema_freeform_asked_learners = schema_freeform[schema_freeform['Asked'] == 'Learners']
schema_freeform_asked_nonswitcher = schema_freeform[schema_freeform['Asked'] == 'Non-switcher']
schema_freeform_asked_onlinelearners = schema_freeform[schema_freeform['Asked'] == 'OnlineLearners']
schema_freeform_asked_worker = schema_freeform[schema_freeform['Asked'].isin(['Worker1','Worker'])]

#### **Define the framework**

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        #print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))


def extract_topics(document, n_topics):
    vect = CountVectorizer(stop_words='english',lowercase= True)
    dtm = vect.fit_transform(document.dropna())
    lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    lda.fit(dtm)
    print_top_words(lda, vect.get_feature_names(), 3)


def calculate_response(col,lst):
    pct = []
    x = pd.DataFrame(data_freeform[col].dropna().str.lower())
    for word in lst:
        total_response = len(x[col])
        p = (len(x[x[col].str.contains(word)])/total_response)*100
        pct.append(p)
    return pd.DataFrame({'Percentage':pct, col:lst}, columns = [col,'Percentage']), total_response

def draw_chart(df):
    df = df[0].sort_values(by = 'Percentage', ascending = False)
    matplotlib.rcParams['figure.figsize'] = (10.0, 4.0)
    plt.bar(np.arange(len(df.iloc[:,0])), df.iloc[:,1], align='center', alpha=0.5)
    plt.xticks(np.arange(len(df.iloc[:,0])), df.iloc[:,0],rotation='vertical')
    plt.ylabel('Percentage')
    plt.title(list(df)[0])
    plt.show()

## **Questions asked to 'All'**

All: Every respondent was shown this question

In [ ]:
schema_freeform_asked_all[['Column','Question']]

###### ****Question : Select your gender identity. - A different identity - Text****
###### ****Column : GenderFreeForm****

In [ ]:
extract_topics(data_freeform['GenderFreeForm'], 5)
calculate_response('GenderFreeForm',['helicopter','attack','apache'])

Good percentage of people identified their gender as **attack helicopter**. Not sure what this mean but not relevant to the analysis.

###### ****Question : Which tool or technology are you most excited about learning in the next year? (Select one option) - Other - Text****
###### ****Column : MLToolNextYearFreeForm****

In [ ]:
extract_topics(data_freeform['MLToolNextYearFreeForm'], 10)
resp = calculate_response('MLToolNextYearFreeForm',['tensorflow','cuda','javascript','keras','scala','haskell','pytorch','h2o','alteryx','azure','cntk','mxnet','latex','pymc3'])
draw_chart(resp)
#type(resp[0])
#resp[0].sort_values(by='Percentage',ascending = False)

 Out of 385 responses received, 16% respondent have written **pytorch**. Other two tools are **keras** and **scala**. None of these responses are present as multiple choice options.

###### ****Question : Which ML/DS method are you most excited about learning in the next year? (Select one option) - Other - Text****
###### ****Column : MLMethodNextYearFreeForm****

In [ ]:
extract_topics(data_freeform['MLMethodNextYearFreeForm'], 20)
resp = calculate_response('MLMethodNextYearFreeForm',['rnn','cnn','natural','gan','reinforcement','gaussian','vision','adversarial','markov','boost',])
draw_chart(resp)

Out of 227 responses received, 24% respondents talked about **reinforcement learning**. 9% talked about **General Adversarial Networks(GANS)**. Needless to say these are the next two big things in AI space.

###### ****Question : What programming language would you recommend a new data scientist learn first? (Select one option) - Other - Text****
###### ****Column : LanguageRecommendationFreeForm****

In [ ]:
extract_topics(data_freeform['LanguageRecommendationFreeForm'], 20)
resp = calculate_response('LanguageRecommendationFreeForm',['wolfram','julia','sql','vb','python','octave','rust','excel','scala','golang','weka'])
draw_chart(resp)

Out of 81 responses received, 17% responded as **Python**.. 6% people suggested **vb**. No doubt, **Octave** and **Rust**, two newly launched robust languages are also in the list.

###### ****Question : Where do you find public datasets to practice data science skills? (Select all that apply) - Other - Text****
###### ****Column : PublicDatasetsFreeForm****


In [ ]:
extract_topics(data_freeform['PublicDatasetsFreeForm'], 20)
#calculate_response('PublicDatasetsFreeForm',['crawl','scrap','collect','reddit','uci','aws','wiki','web'])

Diffrerent public datasources are -

**UCI data repository, Package included datasets, Kaggle, books, webscrapping, aws, Numerai, Uniprot, MSCOCO, Twitter, WIkipedia.**


###### ****Question : What is your biggest challenge with the public datasets you find for personal projects?****
###### ****Column : PersonalProjectsChallengeFreeForm****


In [ ]:
extract_topics(data_freeform['PersonalProjectsChallengeFreeForm'], 20)
resp = calculate_response('PersonalProjectsChallengeFreeForm',['clean','dirty','nois','license','missing','domain','availability'])
draw_chart(resp)

Surprisingly, we have got 3502 responses. Most of the people talked about **clean data, missing values**

###### ****Question : What platforms & resources have you used to continue learning data science skills? (Select all that apply) - Non-Kaggle online communities - Text****
###### ****Column : LearningPlatformCommunityFreeForm****


In [ ]:
extract_topics(data_freeform['LearningPlatformCommunityFreeForm'], 20)
resp = calculate_response('LearningPlatformCommunityFreeForm',['twitter','kdnuggets','qlikview','biostars','opendatascience','facebook','mooc','coursera','vidya','vidhya','analyticsvidhya','reddit','quora','udacity','hackerearth','datatau','datacamp','indico','slack','rbloggers','stackoverflow','quantopian','linkedin','meetups'])
draw_chart(resp)

Althouth response are only 179, many interesting platforms came up. **Reddit** seems to be the leader. Seems, **Analytics Vidhya** doing good job in this area. Other, good platforms are **Coursera, Slack, Datacamp, Quora, Udacity, FB, Opendatascience**. Few not so popular platforms like **Datatau** and **Biostars** has also emerged. People are also using **Meetups** as arranged by some of these platforms (like Analytics Vidhya)for learning and collaborative purposes.

###### ****Question : ****
****What platforms & resources have you used to continue learning data science skills? (Select all that apply) - Other - Text1****
****What platforms & resources have you used to continue learning data science skills? (Select all that apply) - Other - Text2****
****What platforms & resources have you used to continue learning data science skills? (Select all that apply) - Other - Text3****

###### ****Column : ****
****LearningPlatformFreeForm1****
****LearningPlatformFreeForm2****
****LearningPlatformFreeForm3****

In [ ]:
df = pd.DataFrame()
df['LearningPlatformFreeForm'] = data_freeform['LearningPlatformFreeForm1'].dropna().append([data_freeform['LearningPlatformFreeForm2'].dropna(),data_freeform['LearningPlatformFreeForm3'].dropna()])
extract_topics(df['LearningPlatformFreeForm'], 20)
resp1 = calculate_response('LearningPlatformFreeForm1',['edx','book','hackerrank','meetup','analyticsvidhya','vidhya','linkedin','mooc','reddit','google','quora','journals','github','paper','datacamp','udacity','udemy','kagglenoobs','galvanize','youtube','lynda','dataquest'])
resp2 = calculate_response('LearningPlatformFreeForm2',['edx','book','hackerrank','meetup','analyticsvidhya','vidhya','linkedin','mooc','reddit','google','quora','journals','github','paper','datacamp','udacity','udemy','kagglenoobs','galvanize','youtube','lynda','dataquest'])
resp3 = calculate_response('LearningPlatformFreeForm3',['edx','book','hackerrank','meetup','analyticsvidhya','vidhya','linkedin','mooc','reddit','google','quora','journals','github','paper','datacamp','udacity','udemy','kagglenoobs','galvanize','youtube','lynda','dataquest'])
resp = ((resp1[0]['Percentage']*resp1[1]/100 + resp2[0]['Percentage']*resp2[1]/100 + resp3[0]['Percentage']*resp3[1]/100)/(resp1[1] + resp2[1] +resp3[1]))*100
resp = pd.DataFrame(resp1[0]['LearningPlatformFreeForm1']).assign(Percentage = resp)
draw_chart((resp,resp1[1]+resp2[1]+resp3[1]))

Based on total 429 responses received, it **Datacamp, Udacity** happens to be good learning platform. **Meetups** also workings as good channel. 

###### ****Question : What are your top 3 favorite data science blogs/podcasts/newsletters? (Select up to three options) - Other (Separate different answers with semicolon) - Text****
###### ****Column : BlogsPodcastsNewslettersFreeForm****


In [ ]:
extract_topics(data_freeform['BlogsPodcastsNewslettersFreeForm'], 20)
resp = calculate_response('BlogsPodcastsNewslettersFreeForm',['datacamp','udacity','superdatascience','datascienceweekly','data science weekly','analytics vidhya','kaggle','mlwave','hackernoon','indiseai','coursera','udemy'])
draw_chart(resp)

**Analytics Vidhya** stands out from the rest. Others are **data science weekly, super data science.**

###### ****Question : What's the most impactful machine learning algorithm you've ever deployed?****
###### ****Column : ImpactfulAlgorithmFreeForm****


In [ ]:
extract_topics(data_freeform['ImpactfulAlgorithmFreeForm'], 20)
resp = calculate_response('ImpactfulAlgorithmFreeForm',['nda','facial','bayes','decision tree','cluster','apriori','neural','xgboost','gbm','logistic','linear','regression','cnn','knn','randomforest','word2vec','segmentation','svm'])
draw_chart(resp)

###### ****Question : If given the necessary resources and time, what problem would you personally choose to tackle with machine learning?****
###### ****Column : InterestingProblemFreeForm****


In [ ]:
extract_topics(data_freeform['InterestingProblemFreeForm'], 20)

Very interesting to see people thinking to apply ML algorithms to solve these problems -

**Classify news/knowledge, image recognition, cancer research, personalization, behaviour prediction, poverty, education, finance, climate change, healthcare and research, robotics, cryptocurrency, video analytics, security, app building**

###### ****Question : Select all other job titles you've held in the past 10 years? (Select all that apply) - Other - Text****
###### ****Column : PastJobTitlesFreeForm****


In [ ]:
extract_topics(data_freeform['PastJobTitlesFreeForm'], 20)
resp = calculate_response('PastJobTitlesFreeForm',['cto','ceo','accountant','technician','consultant','student','executive','tester','administrator','instructor','admin','reasearcher','linguist','mathematician','econnomist','architect','manager','teacher','actuary','tutor','officer','chief','trader','professor','scientist','designer','bioinformatician','founder','journalist'])
draw_chart(resp)

###### ****Question : How did you first start your machine learning / data science training? (Select one option) - Other - Text****
###### ****Column : FirstTrainingFreeForm****

In [ ]:
extract_topics(data_freeform['FirstTrainingFreeForm'], 20)
resp = calculate_response('FirstTrainingFreeForm',['contest','inbuilt','diploma','self','project','training','udacity','master degree','course','internships','kaggle','book','youtube','knime','bootcamp','certification'])
draw_chart(resp)

Majority of the respondents started with **Courses,Trainings and Bootcamps**. Many are **self taught **used **book, youtube, online courses**.

###### ****Question : In which areas of machine learning do you consider yourself competent? (Select all that apply) - Other (please specify; separate by semi-colon) - Text****
###### ****Column : MLSkillsFreeForm****

In [ ]:
extract_topics(data_freeform['MLSkillsFreeForm'], 20)
resp = calculate_response('MLSkillsFreeForm',['genetic algorithms','hypothesis testing','text mining','predictive modelling','dimension reduction','machine learning','deep learning','time series','regression','statistics','chemometrics','financial engineering','information retrieval','robotics','fuzzy','image processing','signal processing','face recognition','speaker recognition'])
draw_chart(resp)

People think that they are competent in the below areas -

**genetic algorithms, hypothesis testing, text mining, predictive modelling, dimension reduction, machine learning, deep learning, time series, regression, statistics, chemometrics, financial engineering, information retrieval, robotics, fuzzy, image processing, signal processing, face recognition, speaker recognition**


###### ****Question : In which machine learning techniques do you consider yourself competent? (Select all that apply) - Other (please specify; separate by semi-colon) - Text****
###### ****Column : MLTechniquesFreeform****

In [ ]:
extract_topics(data_freeform['MLTechniquesFreeform'], 20)
#calculate_response('MLTechniquesFreeform',[])

Please talked about -

**Meanshift, Association rle mining,TF, NMF, Entropy, Knn, Markov Models, LDA, Gaussian, Optimization, Linear Regression, Opencv, ARIMA, Deep Learning,Oversampling**

## **Questions asked to 'Learners'**

Learners: Respondents who indicated that they were either students, formally or informally learning data science skills, planning to transition into data science, or not employed but looking for work

In [ ]:
schema_freeform_asked_learners[['Column','Question']]

###### **How important do you think the below skills or certifications are in getting a data science job? - Other - Text1**
###### **How important do you think the below skills or certifications are in getting a data science job? - Other - Text2**
###### **How important do you think the below skills or certifications are in getting a data science job? - Other - Text3**

###### ****Column : JobSkillImportanceOtherSelect1FreeForm****
###### ****Column : JobSkillImportanceOtherSelect2FreeForm****
###### ****Column : JobSkillImportanceOtherSelect3FreeForm****

In [ ]:
df = pd.DataFrame()
df['JobSkillImportanceOtherSelect'] = data_freeform['JobSkillImportanceOtherSelect1FreeForm'].dropna().append([data_freeform['JobSkillImportanceOtherSelect2FreeForm'].dropna(),data_freeform['JobSkillImportanceOtherSelect3FreeForm'].dropna()])
extract_topics(df['JobSkillImportanceOtherSelect'], 20)

Responses in relation to interpersonal skills - **Domain knowledge, Thinking skills, Communication skills, passion, patience.**
Responses in relation to Tools/Technologies - **Python, excel, scala, SAS, SPSS, Hadoop, Oracle, Linux, Business Tools, Matlab.**
Responses in relation to demonstrated skill sets - **Data analytics project, work industry experience.**
Responses in relation to degree/course - **Phd, Successfully completed course**

###### ****Question : Which computing hardware do you use for your personal ML/DS projects? - Other - Text****
###### ****Column : HardwarePersonalProjectsFreeForm****

In [ ]:
extract_topics(data_freeform['HardwarePersonalProjectsFreeForm'], 20)
resp=calculate_response('HardwarePersonalProjectsFreeForm',['laptop','cloud','server','desktop','pc'])
draw_chart(resp)

In [ ]:
resp=calculate_response('HardwarePersonalProjectsFreeForm',['hp','dell','lenovo','macbook'])
draw_chart(resp)

In [ ]:
resp=calculate_response('HardwarePersonalProjectsFreeForm',['windows','linux'])
draw_chart(resp)

Obviously, people uses their laptops. Preferably Windows.
However, few people using below for their work -

**1. CSC - Grip GPU**

**2. FloydHub - Cloud offering for Deep Learning (GPU).**

###### ****Question : What's the most important way you can prove your knowledge of ML/DS? (Select one option) - Other - Text****
###### ****Column : ProveKnowledgeFreeForm****

In [ ]:
extract_topics(data_freeform['ProveKnowledgeFreeForm'], 20)

Show case work through -

**Personal projects,Thesis, Good publication, certificate, problem solving, kaggle competitons , developed hobby, conference, project papers. project problems, achievements.**

###### ****Question : Which resource has been the best for finding data science job openings? (Select one option) - Other - Text****
###### ****Column : JobSearchResourceFreeForm****

In [ ]:
extract_topics(data_freeform['JobSearchResourceFreeForm'], 20)
resp=calculate_response('JobSearchResourceFreeForm',['linkedin','network','meetup','analyticsvidya','fb','stackoverfow'])
draw_chart(resp)